# Writing npm (JavaScript) libraries using TypeScript

## Initial Repository Setup

The first thing to do when starting out with creating any new node
package is to initialise a git repo for the package, and create
the `package.json` file.

In [1]:
mkdir -p my-amazing-package

In [2]:
cd my-amazing-package

In [3]:
git init

Initialized empty Git repository in /home/jupyter/notebooks/my-amazing-package/.git/


Next we need to setup the `package.json` file.

Create the file `package.json` with the following content:

```json
{
  "name": "my-amazing-package",
  "version": "0.0.1"
}
```

If you're following along in your own terminal and text editor,
do this as you would any other file.

If you're running the notebook,
the following command will write the file for you.
*(You'll see this pattern throughout the document).*

In [4]:
# Write file
cat > package.json << 'EndOfFile'
{
  "name": "my-amazing-package",
  "version": "0.0.1"
}
EndOfFile

In [5]:
cat package.json

{
  "name": "my-amazing-package",
  "version": "0.0.1"
}


After we have this, we can start installing dependencies for this package.
Let's install the typescript compiler.

*Note: We're going to install this local to the current package, rather than
glocally, to keep things clean*

In [49]:
npm install --save-dev typescript

| |---------------------------------------------------------------------------|
| |---------------------------------------------------------------------------|
- |---------------------------------------------------------------------------|
\ |---------------------------------------------------------------------------|
| |---------------------------------------------------------------------------|
loadRequestedDeps -> get  | |##########---------------------------------------|
loadRequestedDeps -> addN | |##########---------------------------------------|
loadRequestedDeps -> afte \ |##########---------------------------------------|
loadRequestedDeps         \ |###################------------------------------|
runTopLevelLifecycles     - |###############################################--|
npm WARN my-amazing-package@0.0.1 No description
runTopLevelLifecycles     \ |###############################################--|
npm WARN my-amazing-package@0.0.1 No repository field.
runTopLevelLifec

And we can now see how our `package.json` file has changed to include the dependency:

In [50]:
cat package.json

{
  "name": "my-amazing-package",
  "version": "0.0.1",
  "dependencies": {
    "typescript": "^3.5.3"
  },
  "scripts": {
    "tsc": "tsc"
  },
  "devDependencies": {
    "typescript": "^3.5.3"
  }
}


At this point, it's probably a good idea to start checking in files to git.
If we take a look at the `git status`,
we will see however that there are some files we probbly don't want to check-in, namely `node_modules`.

In [8]:
git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	node_modules/
	package.json

nothing added to commit but untracked files present (use "git add" to track)


So let's make sure we're excluding files that we don't want to have checked in to the repo. We can use the starter `.gitignore` file from `resources/`.

Create the file `.gitignore` with the following content:

```
node_modules/
npm-debug.log
```


In [35]:
# Write file
cat > .gitignore << 'EndOfFile'
/node_modules/
/npm-debug.log
EndOfFile

In [36]:
cat .gitignore

/node_modules/
/npm-debug.log


In [11]:
git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.gitignore
	package.json

nothing added to commit but untracked files present (use "git add" to track)


In [12]:
git add .; git commit -m "Initial Commit"

[master (root-commit) d271bbf] Initial Commit
 2 files changed, 9 insertions(+)
 create mode 100644 .gitignore
 create mode 100644 package.json


In [13]:
git status

On branch master
nothing to commit, working tree clean


Make sure you add a remote repository and push regularly too!

## Configuring & compiling TypeScript

Unlike JavaScript, TypeScript can't be run directly in node or in the browser.
We need to first compile it, by using `tsc` to convert the TypeScript code to JavaScript.

This must be done before we publish the package to NPM.

*Note: It's also typically undesirable to include the output of a compilation process
among the checked-in files of a repository. The repository should usually just
contain the sources.*

### Running `tsc`

We need a way to run the command line program `tsc`,
so that we can set up the project and compile the TypeScript code.

However, as we installed the package locally rather than globally,
we can't directly use the `tsc` command in our terminal.

Luckily, you can access locally installed binaries from custom scripts in `package.json`,
and then use them just like normal programs.

Let's modify `package.json` to add the following:

```json
"scripts": {
  "tsc": "tsc"
}
```

In [14]:
json -I -f package.json -e 'this.scripts={tsc:"tsc"}'

json: updated "package.json" in-place


**Note:** In this notebook, we'll be using a command `json` to make required modifications to json files.
But in most situations, you would just edit the file in a text editor.
The json command used here is assumed to be installed with:
```
npm install -g json
```
And if you're running this notebook from within the docker image, `json` should already be installed and available.

In [15]:
cat package.json

{
  "name": "my-amazing-package",
  "version": "0.0.1",
  "dependencies": {
    "typescript": "^3.5.3"
  },
  "scripts": {
    "tsc": "tsc"
  }
}


We can then run `tsc` by instead running `npm run tsc`.

*Note: we use `--` below because all of the command line arguments after `--` are passed directly to the script, and are not processed by node. This is neccesary for some arguments like `--init`*

In [16]:
npm run tsc -- --version


> my-amazing-package@0.0.1 tsc /home/jupyter/notebooks/my-amazing-package
> tsc "--version"

- |---------------------------------------------------------------------------|
Version 3.5.3
- |---------------------------------------------------------------------------|


### Configuring TypeScript

TypeScript is configured using the file `tsconfig.json`.
This file is used both by `tsc`,
and by any integrations in your IDE that aid in TypeScript development.

Let's generate an initial configuration with `tsc --init`.

In [17]:
npm run tsc -- --init


> my-amazing-package@0.0.1 tsc /home/jupyter/notebooks/my-amazing-package
> tsc "--init"

- |---------------------------------------------------------------------------|
message TS6071: Successfully created a tsconfig.json file.
- |---------------------------------------------------------------------------|


In [18]:
cat tsconfig.json

{
  "compilerOptions": {
    /* Basic Options */
    // "incremental": true,                   /* Enable incremental compilation */
    "target": "es5",                          /* Specify ECMAScript target version: 'ES3' (default), 'ES5', 'ES2015', 'ES2016', 'ES2017', 'ES2018', 'ES2019' or 'ESNEXT'. */
    "module": "commonjs",                     /* Specify module code generation: 'none', 'commonjs', 'amd', 'system', 'umd', 'es2015', or 'ESNext'. */
    // "lib": [],                             /* Specify library files to be included in the compilation. */
    // "allowJs": true,                       /* Allow javascript files to be compiled. */
    // "checkJs": true,                       /* Report errors in .js files. */
    // "jsx": "preserve",                     /* Specify JSX code generation: 'preserve', 'react-native', or 'react'. */
    // "declaration": true,                   /* Generates corresponding '.d.ts' file. */
    // "declarationMap": true,                /* Gene

This generates a file with most of the settings commented out and documented, ready for customization.

We want to leave most of these properties at their defaults,
and keep around the documentation for potential use later,
but there are some things that we want to change.

Namely, we want to setup the directories of our repository to look roughly like this:

```
├── lib/ -- output files (.js)
├── src/ -- source files (.ts)
├── package.json
└── tsconfig.json
```

To do this, we want to set the `rootDir` and `outDir` properties, and also add an `include` path. So the tsconfig will look something like this:

```json
{
  "compilerOptions": {
    // other properties
    "outDir": "lib",
    "rootDir": "src"
  },
  "include": [
    "src/**/*"
  ]
}
```

If you're following along and using a text editor,
go ahead and uncomment the appropriate lines and make the relevant changes.

If you're running the notebook,
we need to do something slightly different, because unfortunately,
the `tsconfig.json` file is not actually valid JSON,
as it contains comments,
so we can't use the the `json` command to modify it.

Fortunately though, TypeScript configurations can extend others,
so we can instead create a new file with the options we care about,
and use the generated config as a base:

Let's rename the `tsconfig.json` to `tsconfig.base.json`, and create a new `tsconfig.json` with the following contents:

```json
{
  "extends": "./tsconfig.base",
  "compilerOptions": {
    "outDir": "lib",
    "rootDir": "src"
  },
  "include": [
    "src/**/*"
  ]
}
```

In [19]:
mv tsconfig.json tsconfig.base.json

In [20]:
# Write file
cat > tsconfig.json << 'EndOfFile'
{
  "extends": "./tsconfig.base",
  "compilerOptions": {
    "outDir": "lib",
    "rootDir": "src"
  },
  "include": [
    "src/**/*"
  ]
}
EndOfFile

In [21]:
cat tsconfig.json

{
  "extends": "./tsconfig.base",
  "compilerOptions": {
    "outDir": "lib",
    "rootDir": "src"
  },
  "include": [
    "src/**/*"
  ]
}


Let's see what happens if we try to run the compiler:

In [22]:
npm run tsc


> my-amazing-package@0.0.1 tsc /home/jupyter/notebooks/my-amazing-package
> tsc

- |---------------------------------------------------------------------------|
error TS18003: No inputs were found in config file '/home/jupyter/notebooks/my-amazing-package/tsconfig.json'. Specified 'include' paths were '["src/**/*"]' and 'exclude' paths were '["lib"]'.


Found 1 error.

- |---------------------------------------------------------------------------|

npm ERR! Linux 5.1.15-arch1-1-ARCH
npm ERR! argv "/usr/bin/node" "/usr/bin/npm" "run" "tsc"
npm ERR! node v8.10.0
npm ERR! npm  v3.5.2
npm ERR! code ELIFECYCLE
npm ERR! my-amazing-package@0.0.1 tsc: `tsc`
npm ERR! Exit status 2
npm ERR! 
npm ERR! Failed at the my-amazing-package@0.0.1 tsc script 'tsc'.
npm ERR! Make sure you have the latest version of node.js and npm installed.
npm ERR! If you do, this is most likely a problem with the my-amazing-package package,
npm ERR! not with npm itself.
npm ERR! Tell the author that this fails on your

: 1

It complains that there were no inputs found,
which makes sense as we haven't written any actual TypeScript code yet.

Let's commit the changes we've made so far, and then start writing some TypeScript.

In [23]:
git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   package.json

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	tsconfig.base.json
	tsconfig.json

no changes added to commit (use "git add" and/or "git commit -a")


In [24]:
git add .; git commit -m "Add TypeScript config"

[master 9e34868] Add TypeScript config
 3 files changed, 76 insertions(+)
 create mode 100644 tsconfig.base.json
 create mode 100644 tsconfig.json


In [25]:
git status

On branch master
nothing to commit, working tree clean


## Writing TypeScript

Now that we have set up the compiler options,
let's write our fist typescript file for our package.

Create the folder `src/` and create the file `index.ts` with the following content:

```ts
export function hello_world(a1: string, a2: number){
    console.log(`Hello World! I was given the string ${a1} and the number ${a2}`)
}
```

In [26]:
mkdir src

The following command will write the file for us.
But if you're not running from the notebook, you should create this using a text editor instead.

In [27]:
cat > src/index.ts << 'EndOfTypeScriptFile'
export function hello_world(a1: string, a2: number){
    console.log(`Hello World! I was given the string ${a1} and the number ${a2}`)
}
EndOfTypeScriptFile

In [28]:
cat src/index.ts

export function hello_world(a1: string, a2: number){
    console.log(`Hello World! I was given the string ${a1} and the number ${a2}`)
}


Now if we compile the typescript code:

In [29]:
npm run tsc


> my-amazing-package@0.0.1 tsc /home/jupyter/notebooks/my-amazing-package
> tsc

- |---------------------------------------------------------------------------|
- |---------------------------------------------------------------------------|


We can see that a new directory `lib` has appeared. Inside of which is the file `index.js`.

In [30]:
ls -la

total 44
drwxr-xr-x 6 jupyter jupyter 4096 Jul 16 19:30 .
drwxr-xr-x 5 jupyter jupyter 4096 Jul 16 19:29 ..
drwxr-xr-x 8 jupyter jupyter 4096 Jul 16 19:30 .git
-rw-r--r-- 1 jupyter jupyter   27 Jul 16 19:29 .gitignore
drwxr-xr-x 2 jupyter jupyter 4096 Jul 16 19:30 lib
drwxr-xr-x 4 jupyter jupyter 4096 Jul 16 19:29 node_modules
-rw-r--r-- 1 jupyter jupyter  146 Jul 16 19:29 package.json
drwxr-xr-x 2 jupyter jupyter 4096 Jul 16 19:30 src
-rw-r--r-- 1 jupyter jupyter 5743 Jul 16 19:29 tsconfig.base.json
-rw-r--r-- 1 jupyter jupyter  140 Jul 16 19:29 tsconfig.json


In [31]:
ls -la lib

total 12
drwxr-xr-x 2 jupyter jupyter 4096 Jul 16 19:30 .
drwxr-xr-x 6 jupyter jupyter 4096 Jul 16 19:30 ..
-rw-r--r-- 1 jupyter jupyter  233 Jul 16 19:30 index.js


And if we take a look at that file,
we can see the JavaScript that `tsc` has produced.
All type annotations have been removed, and there are a couple of additions including a `"use strict"` declaration.

In [32]:
cat lib/index.js

"use strict";
Object.defineProperty(exports, "__esModule", { value: true });
function hello_world(a1, a2) {
    console.log("Hello World! I was given the string " + a1 + " and the number " + a2);
}
exports.hello_world = hello_world;


Let's see what the state of our git tree is:

In [33]:
git status

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	lib/
	src/

nothing added to commit but untracked files present (use "git add" to track)


You'll see that both `src/` and `lib/` are listed there as untracked files
that we could potentially check-in to our repository.
But remember earlier that we said that it's preferable to make sure
that compiler output is not checked-in to a repository.

So let's add `/lib/` to `.gitignore`:

In [40]:
echo /lib/ >> .gitignore

In [41]:
cat .gitignore

/node_modules/
/npm-debug.log
/lib/


In [44]:
git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   .gitignore

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	src/

no changes added to commit (use "git add" and/or "git commit -a")


And then let's commit our changes to git.

In [46]:
git add .; git commit -m "Add some typescript code"

[master bdf3a8a] Add some typescript code
 3 files changed, 9 insertions(+), 2 deletions(-)
 create mode 100644 src/.ipynb_checkpoints/index-checkpoint.ts
 create mode 100644 src/index.ts


In [47]:
git status

On branch master
nothing to commit, working tree clean


## Publishing our NPM package

We're almost ready to publish our package to NPM, there are just a couple of things remaining:

* We need to specify which files get included in our package
* We need to specify our `main` module.

### Specifying which files are included when publishing

By default, running `npm publish` will package up every file that's included in your repository
([except for a few special-cases](https://docs.npmjs.com/files/package.json#files)).
This is undesireable as this means that a lot of useless / uneeded files will often be included,
needlessly increasing the size of your published package,
and taking up space in `node_modules` for any project that uses your package as a dependency.

It's in your user's best interest that you keep this as small as possible.

<img src="https://turnoff.us/image/en/npm-install.png" width="400"/>

*Sidenote: There's a great project called [packagephobia](https://packagephobia.now.sh/) that
will calculate the effect that installing an npm package will have on your disk-space.*

There are two primary ways of telling NPM which files to include:

1. blacklisting files using `.npmignore`
2. whitelisting files using the `"files"` property in `package.json`

We're going to use the second option,
as with a whitelist,
we're much less likely to accidentally include new files that are useless.
(for example, docs or files & images that are part of a static site for our project).

We only need to explicitly include the files from `lib/`,
as standard files like `package.json` and `README` or `README.md` are always included
regardless of settings.

So let's add the following to our `package.json`:

```json
"files": [
    "lib/**/*"
]
```

In [51]:
json -I -f package.json -e 'this.files=["lib/**/*"]'

json: updated "package.json" in-place


In [52]:
cat package.json

{
  "name": "my-amazing-package",
  "version": "0.0.1",
  "dependencies": {
    "typescript": "^3.5.3"
  },
  "scripts": {
    "tsc": "tsc"
  },
  "devDependencies": {
    "typescript": "^3.5.3"
  },
  "files": [
    "lib/**/*"
  ]
}


### Specifying our "main" module.

The next thing we need to do is specify which module gets used when users import our package.

I.e., what happens when a user writes:

```js
const amazing = require('my-amazing-package')
```

or

```ts
import * as amazing from 'my-amazing-package'
```

What, should the value of `amazing` be.

For our case, we want this to be the module for the code produced by `src/index.ts`,
which gets output to `lib/index.js`.

So we simply set our value of `"main"` to `"lib/index.js"`:

In [53]:
json -I -f package.json -e 'this.main="lib/index.js"'

json: updated "package.json" in-place


In [54]:
cat package.json

{
  "name": "my-amazing-package",
  "version": "0.0.1",
  "dependencies": {
    "typescript": "^3.5.3"
  },
  "scripts": {
    "tsc": "tsc"
  },
  "devDependencies": {
    "typescript": "^3.5.3"
  },
  "files": [
    "lib/**/*"
  ],
  "main": "lib/index.js"
}


At this point, your package should be ready to publish.
**Make sure that you remember to run `npm run tsc` before publishing if you've made any changes to your TypeScript files**.

You can run `npm publish --dry-run` to see information on what would be published.

Running this command should produce something like this:

```
npm notice 
npm notice 📦  my-amazing-package@0.0.1
npm notice === Tarball Contents === 
npm notice 233B lib/index.js
npm notice 260B package.json
npm notice === Tarball Details === 
npm notice name:          my-amazing-package                      
npm notice version:       0.0.1                                   
npm notice package size:  420 B                                   
npm notice unpacked size: 493 B                                   
npm notice shasum:        7343798e528312644bcd803ca76a3468fefd97ba
npm notice integrity:     sha512-lGijoChg2vnIN[...]Z+S43cYHqsPsg==
npm notice total files:   2                                       
npm notice 
+ my-amazing-package@0.0.1
```

Notice how the only files included are `package.json` and `lib/index.js`.

Once you're happy and ready to publish,
simply run `npm publish`.

You will need to create an account on npmjs.com,
and login using `npm adduser` if you haven't already.
